In [2]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Define the state
class ChatState(TypedDict):
    messages: list

# Initialize the LLM
llm = ChatOllama(
    model="llama3.2",
    temperature=0.7,
)

def chatbot_node(state: ChatState) -> ChatState:
    """Main chatbot node that calls the LLM"""
    messages = state["messages"]
    response = llm.invoke(messages)
    updated_messages = messages + [response]
    return {"messages": updated_messages}

def create_chatbot():
    """Build the chatbot graph"""
    workflow = StateGraph(ChatState)
    workflow.add_node("chatbot", chatbot_node)
    workflow.set_entry_point("chatbot")
    workflow.add_edge("chatbot", END)
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    return app

# Create chatbot instance
chatbot = create_chatbot()
config = {"configurable": {"thread_id": "jupyter_session"}}

# Initialize conversation history
conversation_history = [
    SystemMessage(content="You are a helpful, friendly AI assistant.")
]

def chat(user_message: str):
    """
    Simple chat function for Jupyter notebooks
    Usage: chat("Your message here")
    """
    global conversation_history
    
    # Add user message
    conversation_history.append(HumanMessage(content=user_message))
    
    # Get response
    try:
        state = {"messages": conversation_history}
        result = chatbot.invoke(state, config)
        conversation_history = result["messages"]
        
        # Get AI response
        ai_response = conversation_history[-1].content
        
        # Display conversation
        print(f"You: {user_message}")
        print(f"🤖 Bot: {ai_response}\n")
        
        return ai_response
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\nMake sure:")
        print("1. Ollama is running: ollama serve")
        print("2. Model is installed: ollama pull llama3.2")
        return None

def reset_chat():
    """Reset the conversation history"""
    global conversation_history
    conversation_history = [
        SystemMessage(content="You are a helpful, friendly AI assistant.")
    ]
    print("✅ Conversation reset!")

def show_history():
    """Show the conversation history"""
    print("=" * 60)
    print("CONVERSATION HISTORY")
    print("=" * 60)
    for msg in conversation_history[1:]:  # Skip system message
        if isinstance(msg, HumanMessage):
            print(f"You: {msg.content}")
        elif isinstance(msg, AIMessage):
            print(f"🤖 Bot: {msg.content}")
    print("=" * 60)

In [3]:
chat("Hello! What's your name?")

You: Hello! What's your name?
🤖 Bot: I'm glad you asked. I don't have a personal name in the classical sense, but I'm often referred to as "Assistant" or "AI Assistant" by users like you. If you'd like, I can also generate a nickname or persona for our conversation if it makes things more fun and engaging! Would you like that?



'I\'m glad you asked. I don\'t have a personal name in the classical sense, but I\'m often referred to as "Assistant" or "AI Assistant" by users like you. If you\'d like, I can also generate a nickname or persona for our conversation if it makes things more fun and engaging! Would you like that?'

In [4]:
chat("What can you help me with?")

You: What can you help me with?
🤖 Bot: I'm a versatile AI assistant, so I can help you with a wide range of topics and tasks. Here are some examples:

1. **General knowledge**: I can provide information on various subjects, such as history, science, technology, health, culture, and more.
2. **Language translation**: If you need to translate text or conversation from one language to another, I can help with that too.
3. **Writing assistance**: Whether it's proofreading, suggesting alternative phrases, or even generating content for you, I'm here to support your writing needs.
4. **Conversation and chat**: We can have a fun conversation, play games, or engage in discussions on various topics.
5. **Learning and education**: If you're looking for help with learning a new skill or subject, I can provide guidance, examples, and exercises to get you started.
6. **Brainstorming and ideas**: Sometimes, all you need is someone to bounce ideas off of. I can help generate suggestions and explore d

"I'm a versatile AI assistant, so I can help you with a wide range of topics and tasks. Here are some examples:\n\n1. **General knowledge**: I can provide information on various subjects, such as history, science, technology, health, culture, and more.\n2. **Language translation**: If you need to translate text or conversation from one language to another, I can help with that too.\n3. **Writing assistance**: Whether it's proofreading, suggesting alternative phrases, or even generating content for you, I'm here to support your writing needs.\n4. **Conversation and chat**: We can have a fun conversation, play games, or engage in discussions on various topics.\n5. **Learning and education**: If you're looking for help with learning a new skill or subject, I can provide guidance, examples, and exercises to get you started.\n6. **Brainstorming and ideas**: Sometimes, all you need is someone to bounce ideas off of. I can help generate suggestions and explore different perspectives.\n7. **Jo

In [5]:
chat("Explain AI in simple terms")

You: Explain AI in simple terms
🤖 Bot: I'll try to explain AI in a way that's easy to understand:

**Artificial Intelligence (AI)**: Think of AI as a tool that can think and learn, just like humans do. It uses special computer code to make decisions, recognize patterns, and solve problems.

Imagine you're trying to teach a robot how to play chess. At first, the robot would need to be told every single move it should make to win. But over time, as it plays more games, the robot starts to learn from its mistakes and makes moves on its own. That's basically what AI is doing - learning from data and getting better at tasks over time.

**How does it work?**

There are a few key ways that AI works:

1. **Data**: AI needs lots of information to learn from, like pictures, words, or sounds.
2. **Algorithms**: These are special sets of instructions that tell the AI what to do with the data it's learned.
3. **Computing power**: The more powerful the computer, the faster and more efficient the AI 

"I'll try to explain AI in a way that's easy to understand:\n\n**Artificial Intelligence (AI)**: Think of AI as a tool that can think and learn, just like humans do. It uses special computer code to make decisions, recognize patterns, and solve problems.\n\nImagine you're trying to teach a robot how to play chess. At first, the robot would need to be told every single move it should make to win. But over time, as it plays more games, the robot starts to learn from its mistakes and makes moves on its own. That's basically what AI is doing - learning from data and getting better at tasks over time.\n\n**How does it work?**\n\nThere are a few key ways that AI works:\n\n1. **Data**: AI needs lots of information to learn from, like pictures, words, or sounds.\n2. **Algorithms**: These are special sets of instructions that tell the AI what to do with the data it's learned.\n3. **Computing power**: The more powerful the computer, the faster and more efficient the AI can be.\n\n**Types of AI**